In [1]:
# pip3 install google-cloud-bigquery google-cloud-bigquery-storage pyarrow tqdm ipywidgets pandas matplotlib db-dtypes pandas-gbq

In [2]:
from google.cloud import bigquery

In [3]:
bq = bigquery.Client()

In [11]:
# project => dataset => table
for ds in bq.list_datasets("cs544-f23"):
    print(ds.dataset_id)

test1
test2
test3


In [7]:
ds = bigquery.Dataset("cs544-f23.test3")
# ds.location = "us-central1"
ds

Dataset(DatasetReference('cs544-f23', 'test3'))

In [10]:
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('cs544-f23', 'test3'))

In [12]:
for ds in bq.list_datasets("bigquery-public-data"):
    print(ds.dataset_id)

america_health_rankings
austin_311
austin_bikeshare
austin_crime
austin_incidents
austin_waste
baseball
bbc_news
bigqueryml_ncaa
bitcoin_blockchain
blackhole_database
blockchain_analytics_ethereum_mainnet_us
bls
bls_qcew
breathe
broadstreet_adi
catalonian_mobile_coverage
catalonian_mobile_coverage_eu
census_bureau_acs
census_bureau_construction
census_bureau_international
census_bureau_usa
census_opportunity_atlas
census_utility
cfpb_complaints
chicago_crime
chicago_taxi_trips
clemson_dice
cloud_storage_geo_index
cms_codes
cms_medicare
cms_synthetic_patient_data_omop
country_codes
covid19_aha
covid19_covidtracking
covid19_ecdc
covid19_ecdc_eu
covid19_genome_sequence
covid19_geotab_mobility_impact
covid19_geotab_mobility_impact_eu
covid19_google_mobility
covid19_google_mobility_eu
covid19_govt_response
covid19_italy
covid19_italy_eu
covid19_jhu_csse
covid19_jhu_csse_eu
covid19_nyt
covid19_open_data
covid19_open_data_eu
covid19_public_forecasts
covid19_public_forecasts_asia_ne1
covid19_r

In [13]:
for t in bq.list_tables("bigquery-public-data.github_repos"):
    print(t.table_id)

commits
contents
files
languages
licenses
sample_commits
sample_contents
sample_files
sample_repos


In [14]:
%load_ext google.cloud.bigquery

In [15]:
%%bigquery
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"
5,TianYJ1/Snake,"[{'name': 'C', 'bytes': 10381}]"
6,doctorfree/Lds,"[{'name': 'C', 'bytes': 56413}]"
7,SumiMakito/ReinaJNIDemo,"[{'name': 'C', 'bytes': 7444}]"
8,serprex/rainbeam,"[{'name': 'C', 'bytes': 1964}]"
9,pdigiglio/oracolo,"[{'name': 'C', 'bytes': 1966}]"


In [16]:
%%bigquery df
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
df

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"
5,TianYJ1/Snake,"[{'name': 'C', 'bytes': 10381}]"
6,doctorfree/Lds,"[{'name': 'C', 'bytes': 56413}]"
7,SumiMakito/ReinaJNIDemo,"[{'name': 'C', 'bytes': 7444}]"
8,serprex/rainbeam,"[{'name': 'C', 'bytes': 1964}]"
9,pdigiglio/oracolo,"[{'name': 'C', 'bytes': 1966}]"


In [28]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [29]:
q = bq.query("""
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 10
""", job_config=no_cache)

In [30]:
df = q.to_dataframe()
df

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"
5,TianYJ1/Snake,"[{'name': 'C', 'bytes': 10381}]"
6,doctorfree/Lds,"[{'name': 'C', 'bytes': 56413}]"
7,SumiMakito/ReinaJNIDemo,"[{'name': 'C', 'bytes': 7444}]"
8,serprex/rainbeam,"[{'name': 'C', 'bytes': 1964}]"
9,pdigiglio/oracolo,"[{'name': 'C', 'bytes': 1966}]"


In [31]:
q.total_bytes_processed / 1024**2 # MB

194.93627738952637

In [32]:
q.total_bytes_billed / 1024**2 # MB

195.0

In [ ]:
# how many times can we do this query for free (per month)?

In [33]:
free_tier = 1024**4 # 1 TB
free_tier / q.total_bytes_billed

5377.3128205128205

In [ ]:
# how much does it cost in Tokyo after we leave the free tier?

In [34]:
tokyo_cost = 7.5 # per TB
q.total_bytes_billed / 1024**4 * tokyo_cost

0.0013947486877441406

In [35]:
# cost factors:
# 1. pay for storage space (in addition to queries)
# 2. round up to nearest 1 MB (per query)
# 3. min of 10 MB (per query)

In [37]:
%%bigquery
SELECT *
FROM `bigquery-public-data.github_repos.languages`
WHERE repo_name LIKE 'open-lambda/%'
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,repo_name,language
0,open-lambda/open-lambda,"[{'name': 'C', 'bytes': 31159}, {'name': 'Dock..."
1,open-lambda/load-balancer,"[{'name': 'C', 'bytes': 210}, {'name': 'Go', '..."
